In [1]:
import pandas as pd

# Load the CSV dataset
df = pd.read_csv("train_timeseries.csv")

# Quick look
print(df.head())
print(df.info())
print(df.describe())


   fips        date  PRECTOT      PS   QV2M    T2M  T2MDEW  T2MWET  T2M_MAX  \
0  1001  2000-01-01     0.22  100.51   9.65  14.74   13.51   13.51    20.96   
1  1001  2000-01-02     0.20  100.55  10.42  16.69   14.71   14.71    22.80   
2  1001  2000-01-03     3.65  100.15  11.76  18.49   16.52   16.52    22.73   
3  1001  2000-01-04    15.95  100.29   6.42  11.40    6.09    6.10    18.09   
4  1001  2000-01-05     0.00  101.15   2.95   3.86   -3.29   -3.20    10.82   

   T2M_MIN  ...     TS  WS10M  WS10M_MAX  WS10M_MIN  WS10M_RANGE  WS50M  \
0    11.46  ...  14.65   2.20       2.94       1.49         1.46   4.85   
1    12.61  ...  16.60   2.52       3.43       1.83         1.60   5.33   
2    15.32  ...  18.41   4.03       5.33       2.66         2.67   7.53   
3     2.16  ...  11.31   3.84       5.67       2.08         3.59   6.73   
4    -2.66  ...   2.65   1.60       2.50       0.52         1.98   2.94   

   WS50M_MAX  WS50M_MIN  WS50M_RANGE  score  
0       6.04       3.23     

In [2]:
print(df.isnull().sum())


fips                  0
date                  0
PRECTOT               0
PS                    0
QV2M                  0
T2M                   0
T2MDEW                0
T2MWET                0
T2M_MAX               0
T2M_MIN               0
T2M_RANGE             0
TS                    0
WS10M                 0
WS10M_MAX             0
WS10M_MIN             0
WS10M_RANGE           0
WS50M                 0
WS50M_MAX             0
WS50M_MIN             0
WS50M_RANGE           0
score          16543884
dtype: int64


In [3]:
df.fillna(method='ffill', inplace=True)  # forward fill
df.fillna(method='bfill', inplace=True)  # backward fill


C:\Users\ASUS\AppData\Local\Temp\ipykernel_21128\1028311457.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)  # forward fill
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21128\1028311457.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)  # backward fill


In [4]:
# Example: 3-day rolling averages
df['PRECTOT_3d'] = df['PRECTOT'].rolling(3).mean()
df['T2M_MAX_3d'] = df['T2M_MAX'].rolling(3).mean()
df['T2M_MIN_3d'] = df['T2M_MIN'].rolling(3).mean()
df['WS10M_3d'] = df['WS10M'].rolling(3).mean()

# Fill NaNs created by rolling
df.fillna(method='bfill', inplace=True)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_21128\3475226194.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [5]:
df['PRECTOT_diff'] = df['PRECTOT'] - df['PRECTOT'].shift(1)
df['T2M_diff'] = df['T2M'] - df['T2M'].shift(1)
df.fillna(0, inplace=True)


In [6]:
from sklearn.model_selection import train_test_split

features = [col for col in df.columns if col not in ['fips','date','score']]
X = df[features]
y = df['score']

# Train-test split (time-series aware)
train_size = int(len(df)*0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [8]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

y_pred = model.predict(X_test)

# RMSE manually
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

# R2
print("R2:", r2_score(y_test, y_pred))


RMSE: 1.1832616952439736
R2: 0.16934006581753647


In [15]:
import optuna
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from math import sqrt
import pandas as pd

# Example: Replace with your actual training data
# X_train, y_train = ...
# Make sure X_train is a DataFrame and y_train is a Series

def objective(trial):
    # Hyperparameter search space
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
        "random_state": 42,
        "tree_method": "hist",   # Faster training
        "n_jobs": -1             # Use all CPU cores
    }
    
    tscv = TimeSeriesSplit(n_splits=3)  # reduce folds for speed
    rmse_list = []
    
    for train_idx, val_idx in tscv.split(X_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model = XGBRegressor(**param)
        model.fit(X_tr, y_tr)  # ⚡ simpler call, no unsupported args
        
        y_pred = model.predict(X_val)
        rmse = sqrt(mean_squared_error(y_val, y_pred))
        rmse_list.append(rmse)
    
    return sum(rmse_list) / len(rmse_list)

# Run Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20, n_jobs=1)

print("Best RMSE:", study.best_value)
print("Best hyperparameters:", study.best_params)


[I 2025-08-25 17:18:33,714] A new study created in memory with name: no-name-ed33f9b5-4870-409a-9b5f-c3bc12494923
[I 2025-08-25 17:27:19,041] Trial 0 finished with value: 1.0145878562673059 and parameters: {'n_estimators': 429, 'max_depth': 11, 'learning_rate': 0.030143900853127858, 'subsample': 0.9190878894427204, 'colsample_bytree': 0.6853003663519037, 'gamma': 2.7152423362075533, 'reg_alpha': 0.34477974840683434, 'reg_lambda': 0.4194221257868749}. Best is trial 0 with value: 1.0145878562673059.
[I 2025-08-25 17:29:52,393] Trial 1 finished with value: 1.0231095923048106 and parameters: {'n_estimators': 192, 'max_depth': 8, 'learning_rate': 0.12214654544836807, 'subsample': 0.8575555870700624, 'colsample_bytree': 0.6188413978301277, 'gamma': 0.6306674239824667, 'reg_alpha': 0.29674377773298755, 'reg_lambda': 0.22571303041408297}. Best is trial 0 with value: 1.0145878562673059.
[I 2025-08-25 17:33:12,351] Trial 2 finished with value: 1.0498653716921995 and parameters: {'n_estimators': 

Best RMSE: 1.0114867134202632
Best hyperparameters: {'n_estimators': 373, 'max_depth': 12, 'learning_rate': 0.08149271478740591, 'subsample': 0.9947387447060678, 'colsample_bytree': 0.9754287046543713, 'gamma': 4.9673693977484055, 'reg_alpha': 0.5135635463977105, 'reg_lambda': 0.8430828445040679}


In [16]:
import pickle
from xgboost import XGBRegressor

# Train with best params
best_model = XGBRegressor(**study.best_params)
best_model.fit(X_train, y_train)

# Save model
with open("drought_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("Model saved as drought_model.pkl")


Model saved as drought_model.pkl
